In [128]:
import pandas as pd
import numpy as np

#### Settings

In [129]:
pd.set_option('max_colwidth', 200)

### EDA

In [130]:
poems = pd.read_csv('data/OCPoetry.csv', index_col = 0)
sss = pd.read_csv('data/shortscarystories.csv', index_col=0)

In [131]:
poems['subreddit'] = 'OCPoetry'
sss['subreddit'] = 'shortscarystories'

In [132]:
poems.head()

,title,text,auth,time,subreddit
0,the moths of time,the moths of time consume your image; everything of you is gone from my life.\nbut after all this time i still glimpse your grimace in a mirror that reflects when the world was rife.\n\n\nhand on ...,lizerdqweenchlo,1643240027,OCPoetry
1,Haunted Houses,Floorboards creak\n\nUnder little kid feet\n\nAs a new family arrives\n\nThat the siblings want to meet\n\n&amp;#x200B;\n\nA big sister!\n\nA big brother!\n\nA new father!\n\nAnd new mother!\n\n&a...,richardcrack,1643238939,OCPoetry
2,Forest of Eden,"&amp;#x200B;\n\nHe could never quite find\n\nWhat made it paradise. \n\nWas it the way needles fell \n\nFrom the shadeless pines, \n\n&amp;#x200B;\n\nOr the river that raised \n\nHis grandmothers’...",mgmgmgmgm,1643237039,OCPoetry
3,The deepest fluctuation of creativity,"With due regard at your behest\n\nI'll smear the ink and burn the rest,\n\na long and overdue repose\n\nwith words withheld and all books closed,\n\nI'll lie as silent as the grave\n\nand sleep in...",puredreadful,1643234027,OCPoetry
4,The deepest fluctuation of creativity,"With due regard at your behest\n\nI'll smear the ink and burn the rest,\n\na long and overdue repose\n\nwith words withheld and all books closed,\n\nI'll lie as silent as the grave\n\nand sleep in...",puredreadful,1643234027,OCPoetry


In [133]:
poems.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29905 entries, 0 to 48
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   title      29905 non-null  object
 1   text       29905 non-null  object
 2   auth       29905 non-null  object
 3   time       29905 non-null  int64 
 4   subreddit  29905 non-null  object
dtypes: int64(1), object(4)
memory usage: 1.4+ MB


In [134]:
sss.head()

,title,text,auth,time,subreddit
0,Fruit of Human Kindness,"**Let it be stated, for the record, that what follows is the examination of one Conrad Marlow, in relation to allegations made against Mr Marlow's former employer, the anthropologist and sitting M...",normancrane,1643239674,shortscarystories
1,MEDICINE BALL,"\n\nMy boyfriend and I felt we were not getting exercise and outdoors enough during this bleak winter. Even before the pandemic neither of us were active enough, and with both working long hours...",eloquentursula,1643229661,shortscarystories
2,The Mandela Effect Is Real And I can Prove it.,\n\nDoes anybody remember the film The Lost Boys? It was my favourite film as a kid. I idolised the Frog Brothers and imagined myself fighting vampires side by side with them. I had an awesome p...,Roos85,1643220793,shortscarystories
3,The most weird call I had ever gotten as a babysitter. (Fictional),"Hi, Im Natalie, 24 year old babysitter, and I wanted to share one of my most bone chilling experiences with a costumer.\n\nIt was around 4am, I was deep asleep, snoring so loud it sounded like a ...",Riaxxxxxx_,1643220655,shortscarystories
4,Forever young,"They will tell you she killed herself.\n\nShe didn't. I saw what happened, I told and explained them.\n\nThey didn't believe me. Said that I need rest, that I was in denial, that my mind played tr...",Thunder-and-Dark,1643213868,shortscarystories


In [135]:
sss.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28687 entries, 0 to 93
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   title      28687 non-null  object
 1   text       28586 non-null  object
 2   auth       28687 non-null  object
 3   time       28687 non-null  int64 
 4   subreddit  28687 non-null  object
dtypes: int64(1), object(4)
memory usage: 1.3+ MB


In [136]:
df = pd.concat([poems, sss])
df.reset_index(drop=True, inplace=True)

In [137]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58592 entries, 0 to 58591
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   title      58592 non-null  object
 1   text       58491 non-null  object
 2   auth       58592 non-null  object
 3   time       58592 non-null  int64 
 4   subreddit  58592 non-null  object
dtypes: int64(1), object(4)
memory usage: 2.2+ MB


### Data Cleaning

#### Fill empty strings

In [138]:
df['text'].fillna('', inplace=True)

#### Count whitespace

Whitespace features could be predictive of poetry - I will count them here and then remove them from the text, so they can be included or not in later analysis.

In [139]:
df['new_line_chars'] = df['text'].str.count('\n')
# df['tab_chars'] = df['text'].str.count('\t') Not needed as very few tabs
df['space_chars'] = df['text'].str.count(' ')

In [140]:
df['space_chars'].value_counts()

0       1338
490      295
487      283
485      279
482      278
        ... 
3720       1
2730       1
792        1
945        1
5313       1
Name: space_chars, Length: 1368, dtype: int64

In [141]:
df.head()

,title,text,auth,time,subreddit,new_line_chars,space_chars
0,the moths of time,the moths of time consume your image; everything of you is gone from my life.\nbut after all this time i still glimpse your grimace in a mirror that reflects when the world was rife.\n\n\nhand on ...,lizerdqweenchlo,1643240027,OCPoetry,22,134
1,Haunted Houses,Floorboards creak\n\nUnder little kid feet\n\nAs a new family arrives\n\nThat the siblings want to meet\n\n&amp;#x200B;\n\nA big sister!\n\nA big brother!\n\nA new father!\n\nAnd new mother!\n\n&a...,richardcrack,1643238939,OCPoetry,164,253
2,Forest of Eden,"&amp;#x200B;\n\nHe could never quite find\n\nWhat made it paradise. \n\nWas it the way needles fell \n\nFrom the shadeless pines, \n\n&amp;#x200B;\n\nOr the river that raised \n\nHis grandmothers’...",mgmgmgmgm,1643237039,OCPoetry,42,69
3,The deepest fluctuation of creativity,"With due regard at your behest\n\nI'll smear the ink and burn the rest,\n\na long and overdue repose\n\nwith words withheld and all books closed,\n\nI'll lie as silent as the grave\n\nand sleep in...",puredreadful,1643234027,OCPoetry,16,34
4,The deepest fluctuation of creativity,"With due regard at your behest\n\nI'll smear the ink and burn the rest,\n\na long and overdue repose\n\nwith words withheld and all books closed,\n\nI'll lie as silent as the grave\n\nand sleep in...",puredreadful,1643234027,OCPoetry,16,34


#### Filter URLs

URLs are difficult to do NLP on and they are also far more common in the OCPoetry subreddit which biases our models. To fix this we should remove them from the text column.

In [142]:
df.query('subreddit == "OCPoetry"')['text'].sample()

11188    She left as souls leave,\n\nGentle and easy;\n\nOne could not see from\n\nOutside the body,\n\nOutlined on concrete,\n\nThe pain he still keeps.\n\nhttps://www.reddit.com/r/OCPoetry/comments/ni4tr...
Name: text, dtype: object

In [143]:
df.loc[23854, 'text'] # Example post with 2 urls

'\nwrite songs for my poems\nput method to my madness\nplease deaden my mind\n\n[1](https://www.reddit.com/r/OCPoetry/comments/ir71un/lusts_smile/?utm_source=share&amp;utm_medium=ios_app&amp;utm_name=iossmf)\n\n[2](https://www.reddit.com/r/OCPoetry/comments/ipgn9o/a_dream_i_once_had/?utm_source=share&amp;utm_medium=ios_app&amp;utm_name=iossmf)'

In [144]:
df.loc[23854, 'text'].split()

['write',
 'songs',
 'for',
 'my',
 'poems',
 'put',
 'method',
 'to',
 'my',
 'madness',
 'please',
 'deaden',
 'my',
 'mind',
 '[1](https://www.reddit.com/r/OCPoetry/comments/ir71un/lusts_smile/?utm_source=share&amp;utm_medium=ios_app&amp;utm_name=iossmf)',
 '[2](https://www.reddit.com/r/OCPoetry/comments/ipgn9o/a_dream_i_once_had/?utm_source=share&amp;utm_medium=ios_app&amp;utm_name=iossmf)']

In [145]:
[token.find('https') for token in df.loc[23854, 'text'].split()]

[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 4, 4]

In [146]:
' '.join([token for token in df.loc[23854, 'text'].split() if token.find('https') == -1])

'write songs for my poems put method to my madness please deaden my mind'

In [147]:
def scrub_url(text):
    return ' '.join([token for token in text.split() if token.find('https') == -1])

In [148]:
df['text'] = [scrub_url(text) for text in df['text']]

In [149]:
df['text'].str.count('https').sum() # check scrubbing

0

#### Count words

In [150]:
df['title_words'] = df['title'].str.split().str.len()

In [151]:
df['title_words'].value_counts()

2     13346
1     11034
3     10396
4      7509
5      5599
6      3509
7      2084
8      1381
9       995
10      640
11      442
12      348
13      251
14      217
15      142
16      132
17       82
19       60
18       60
20       43
22       41
21       32
23       26
24       25
25       18
26       15
28       12
29       12
32       12
30       11
44       10
34       10
27       10
31        9
49        8
35        7
33        7
42        6
48        5
39        5
40        5
37        4
36        4
38        4
52        3
47        3
41        2
53        2
59        2
57        2
60        2
63        1
54        1
43        1
56        1
51        1
45        1
46        1
50        1
Name: title_words, dtype: int64

In [152]:
df.sort_values(by = 'title_words', ascending=False)

,title,text,auth,time,subreddit,new_line_chars,space_chars,title_words
38464,I just want to die i cant live in this house with them anymore just because i don’t want to leave my room she got 7 other CHILD why does it have to be me why do i have to do the chores why does sh...,Please if they didnt they then i wish i will,S9_b10,1612031128,shortscarystories,0,9,63
38213,"Hola , personas de Reddit el otro día recordé una película que ví hace muchos años que tenía un corto asiático de terror acerca de un chico que hace un ritual y acaba en otra dimensión muy bizarra...",,Key-frank-3678,1613326250,shortscarystories,0,0,60
30893,I heard this of this short story about a man who spends his night asking a girl who comes out her door and she feels forced into doing it by this guy verbally abusing her outside her door. She eve...,,carlpoppa618II,1639616991,shortscarystories,0,0,60
20427,"Title: COVID-19. I wrote this a little while back, and have just found this sub reddit. Idk the culture here, so this is gonna be just how I write and recite. I try to break it in ways that can be...",Front lines have become local And we got new soldiers In scrubs and white coats and They trying to save the lives of the people Still low on the PPE for the professionals How come THEY have it una...,chill-_-monkey,1606624927,OCPoetry,26,97,59
26821,"Hello. I'm Santimental, but you can call me just San. I'm usually a story writer, but still a storyteller. I found some muse for writing, and defining it as my poetry was the closest way for me to...","My meaning? Soul. Not purpose. Meaning. The thing, striving after which things make just that bit more sense. It satisfies that itch for something that doesn't exist. How do you make meaning, soul...",Santimental,1593677442,OCPoetry,4,486,59
...,...,...,...,...,...,...,...,...
27359,Tonight,"Where are you now? Have you seen the moon tonight? I am standing here, with my heart, waiting for you tonight. &amp;#x200B; Tears have filled your eyes, I can see, my love, I’m here. My home, that...",hammadahmedj,1592433137,OCPoetry,58,158,1
27357,Threads,We looked down My fourteenth floor balcony You talked about the endless traffic How the shades and the stereo Create a small secluded space Within each hurrying hatchback How irrelevant were The r...,kaccha_kela,1592434330,OCPoetry,76,493,1
27356,Fade-Away,I could sing The saddest song Of how the stars were stolen Leaving a void in the sky How they’ll never again shine. I could write The saddest story Of the aging moon falling Disappearing into the ...,drak0ni,1592437202,OCPoetry,52,71,1
44503,Clapping,"Somewhere, in the middle of the world, a married couple went on a hike. They stayed until the sun started to settle behind the horizon, but it was too late when they realized they dont know the wa...",InsulineCoffeeAddict,1593429745,shortscarystories,46,285,1


In [153]:
df['text_words'] = df['text'].str.split().str.len()
df.sort_values(by = 'text_words', ascending=False)

,title,text,auth,time,subreddit,new_line_chars,space_chars,title_words,text_words
34902,Desert Nights,"&amp;#x200B; I was five years old when I first met him. He stood at the end of our gravel driveway, sitting back on his heels as I sat on the bumper of my father’s Oldsmobile. My fingers were wedg...",jerseynay,1626303728,shortscarystories,301,7023,2,6935
37660,Howl,The heat of the fireplace warms me up while I hear my companions breathing around. The fire brings back the memories of how everything changed when the gaze of the beast set me free. I must start ...,rmanenti,1615930513,shortscarystories,274,6724,1,6860
53154,Rocky's Ride,"Floyd Rockner, better known as Rocky to the few friends he had, was coming home from a long day of fishing and an even longer day of drinking. He had left his house, no more than an over glorified...",a_boy_named_sir,1557878404,shortscarystories,232,6821,2,6852
47456,The Golden Age of Humanity,Greetings my fellow people! &amp;#x200B; This is a message to all of humanity from the Universe itself. The Collective Consciousness has been unified and as such I have prepared this brief update ...,jiggyfashizzy369,1583919981,shortscarystories,166,5551,5,5634
37038,I've discovered the first black dwarf star... (Part 1),"One of the most universal experiences within humanity are the moments spent gazing into the cosmic time map of the stars. Many have wondered if other life exists, how did it all begin, what does t...",_thelonewolfe_,1619282591,shortscarystories,176,5530,9,5575
...,...,...,...,...,...,...,...,...,...
41945,INFERNO,,3bery,1601131090,shortscarystories,0,0,1,0
14088,Poem lll,,afieldofdaffodils51,1615980836,OCPoetry,1,0,2,0
41941,Scary Saturdays with Arkadeep,,gezelex,1601135882,shortscarystories,0,0,4,0
26888,THE BROKEN PARADISE,,Lennyongori,1593520338,OCPoetry,0,0,3,0


In [154]:
df.query('text_words == 1')

,title,text,auth,time,subreddit,new_line_chars,space_chars,title_words,text_words
71,Cheeks,[deleted],[deleted],1643122296,OCPoetry,0,0,1,1
335,TEST,TEST,ParadiseEngineer,1642620761,OCPoetry,0,0,1,1
847,Reflection in water,[deleted],[deleted],1641741301,OCPoetry,0,0,3,1
1246,"New year, new me?",[deleted],[deleted],1640996990,OCPoetry,0,0,4,1
1283,A Man With A Gun In The Closet,[deleted],[deleted],1640914118,OCPoetry,0,0,8,1
...,...,...,...,...,...,...,...,...,...
57279,I am not Suicidal,[deleted],[deleted],1533905293,shortscarystories,0,0,4,1
57280,Overthinking,[deleted],[deleted],1533886830,shortscarystories,0,0,1,1
57316,Happy Birthday Hannah,\[removed\],Ghettoceratops,1533749725,shortscarystories,0,0,3,1
57318,Carolines Notebooks,\[removed\],minroe-mafdet,1533748131,shortscarystories,0,0,2,1


In [155]:
df.query('text_words == 5')

,title,text,auth,time,subreddit,new_line_chars,space_chars,title_words,text_words
3158,Death | An Original Poem,Death | An Original Poem,k_e_francis,1637340193,OCPoetry,2,5,5,5
3503,Snelvin Schneedles,Check out Dan Kulifay's poem!,breakfasticecream,1636625994,OCPoetry,2,4,2,5
5292,For high school students,poetry contest to join link,wertu1221,1632865940,OCPoetry,0,5,4,5
12216,Jensup,imperlocutability was not to be,beumuth,1619990173,OCPoetry,2,5,1,5
12219,Jensup,impercutability &amp;nbsp;&amp;nbsp;&amp;nbsp;&amp;nbsp;&amp;nbsp;&amp;nbsp;&amp;nbsp;&amp;nbsp;&amp;nbsp;^(&amp;nbsp;&amp;nbsp;)was not to be,beumuth,1619988610,OCPoetry,2,5,1,5
13370,Accumulation,"I, it: pit, spit, spite.",nowreefill,1617892683,OCPoetry,12,0,1,5
14072,A Reminder that Life is Beautiful.,Share your thoughts on youtube!,presentedbyjay,1615999439,OCPoetry,2,4,6,5
15634,My Shortest Poem....and hardest to write,**Stare** Think............................. *Type* Read Feel,needa-Peace-4fs,1613451430,OCPoetry,12,4,6,5
17420,Spoken Word Poetry - He Said,[He Said - Spoken Word,rowe_asia,1610984003,OCPoetry,0,5,6,5
19306,The Code of Saint Incy's.,Spider Poem! M :) X,MarkDPrice,1608240205,OCPoetry,6,5,5,5


In [156]:
df.query('text_words == 8')

,title,text,auth,time,subreddit,new_line_chars,space_chars,title_words,text_words
3111,Two,The theremin and the bazooka touched ever so,nowreefill,1637427837,OCPoetry,4,7,1,8
4577,Distance,A Love letter sent across the country. Unopened.,BennyBoy532,1634506519,OCPoetry,6,10,1,8
6294,Spoken Word,You Will Know Its Me | Spoken Word,Important-Assist1281,1630764482,OCPoetry,0,8,2,8
6295,Check out piem,You Will Know Its Me | Spoken Word,Important-Assist1281,1630764432,OCPoetry,0,8,3,8
7455,Plants,Branches untucked in mobile hermeticity writhe joints loose.,beumuth,1628558037,OCPoetry,6,6,1,8
7456,Plant,Branches untucked in mobile hermeticity writhe joints loose.,beumuth,1628557826,OCPoetry,6,6,1,8
7945,Vocab by Sample Sentence: Agraffe,**Agraffe** (noun) &gt;Paragraph-paraphing pariah's graphite-scuffed giraffe agraffe detached.,beumuth,1627622794,OCPoetry,4,7,5,8
12212,Phial,A phial of to symbolize I'm there for,beumuth,1620000895,OCPoetry,2,8,1,8
13667,"Hello, I got my new opensignal account",I hope you like them [OrderlyMessy's account |,colderlarry,1617470937,OCPoetry,0,8,7,8
15541,Naked Soul - Devotional Love Poems,Thought you might enjoy some of these poems,Agreeable_Pay_772,1613660412,OCPoetry,2,7,6,8


In [157]:
df.query('text_words == 15')

,title,text,auth,time,subreddit,new_line_chars,space_chars,title_words,text_words
62,haiku no.2,"&amp;#x200B; regardless regime, i often make my room a... fridge. for superman! &amp;#x200B; 1. 2.",holy-pizza-tree,1643136321,OCPoetry,16,8,2,15
486,Frog in a Summer Storm,I sleep soundly here Tucked between my river bed And warm sheets of rain &amp;,LapsusCalamii,1642371404,OCPoetry,8,13,5,15
604,3.,"athens is too cold. tonight. and all my friends, are ten years away. 1. 2.",holy-pizza-tree,1642167533,OCPoetry,14,10,1,15
1085,Dakini Requiem,Gathering around The still corpse of someone loved Merry buzzards feast Feedback Currency: (1) (2),SevenBillionBuddhas,1641284259,OCPoetry,11,14,2,15
2181,transient thoughts,cars drive by on the wet road. passing by like fleeting recollections of hazy memories.,Nice-ish-guy,1639107701,OCPoetry,10,11,2,15
...,...,...,...,...,...,...,...,...,...
56022,Dead words,"My daughter talks to me every night, she must have learned to after the funeral.",Boonian,1541088107,shortscarystories,0,14,2,15
56027,I’m home alone.,I just felt something at the edge of my bed touch my foot. I’m alone.,YaoiFanatic,1541049042,shortscarystories,0,15,3,15
56614,b,Listen to the rain falling Soft like petals on the grass Growing over her grave.,bottlerocket23,1537769852,shortscarystories,2,16,1,15
57039,New York serial killer documentary | No One Is Safe From Son of Sam,"Watch this suspense video!! &amp;#x200B; Kindly subscribe this channel. Also like, share and comment it.",funkyfitvirtual,1534933514,shortscarystories,4,12,14,15


In [158]:
# Short posts are often just a link since we are interested in NLP we will drop them
df.query('text_words > 8', inplace = True)

#### Check for duplicates

I'll drop all the duplicate entries.

In [159]:
df.duplicated(subset = ['text', 'title', 'subreddit']).sum()

734

In [160]:
df.drop_duplicates(subset = ['text', 'title', 'subreddit'], inplace=True)

In [161]:
df['subreddit'].value_counts()

OCPoetry             28554
shortscarystories    27768
Name: subreddit, dtype: int64

In [162]:
df.to_csv('data/data_cleaned.csv', index = False)